In [126]:
#This cell is used to import modules that will be useful for model prediction. 

import pandas as pd #Pandas is the base Python library used to manuipulate datasets, typically this will always be useful. 
import numpy as np #Numpy is mainly used to allow for mathematical manipulations of datasets, typically this will always be useful.
import seaborn as sns



In [127]:
#Importing datasets
df1 = pd.read_csv('Datasets/salarysample.csv', usecols=['Job Title','Avg Salary(K)','Job Description','Location']) #Used to read .csv files.
df2=pd.read_excel('Datasets/US_Population_2020_2021.xlsx', sheet_name='2021') #Used to read .xls files, sheet_name can be used to select for different sheets by using index number or string name.
df2 #Initial dataset without any manipulation.

,Geographic Area,Population
0,"New York city, New York",8467513
1,"Los Angeles city, California",3849297
2,"Chicago city, Illinois",2696555
3,"Houston city, Texas",2288250
4,"Phoenix city, Arizona",1624569
...,...,...
799,"Cypress city, California",49926
800,"Murray city, Utah",49729
801,"Chesterfield city, Missouri",49703
802,"Downers Grove village, Illinois",49654


In [128]:
df1['Location'] = df1['Location'].apply(lambda comma: comma.split(',')[0])

df2['Geographic Area']=df2['Geographic Area'].map(lambda symbol : symbol.replace(' city',''))
df2['Geographic Area'] = df2['Geographic Area'].apply(lambda comma: comma.split(',')[0])
df2['Location']=df2['Geographic Area']
df3=pd.merge(df1, df2, on='Location')
df3['Salary']=df3['Avg Salary(K)']*1000

job_description_list= []

for row in df3['Job Title']:
    if 'Scientist' in row:
        job_description_list.append('Data Scientist')
    elif 'Engineer' in row:
        job_description_list.append('Data Engineer')
    elif 'Analyst' in row:
        job_description_list.append('Data Analyst')
    else:
        job_description_list.append('Other')
df3['Categorized Job Title'] = job_description_list



df3.drop(columns={'Geographic Area','Avg Salary(K)','Job Description','Location','Job Title'},inplace=True)
df3

,Population,Salary,Categorized Job Title
0,562599,72000.0,Data Scientist
1,562599,75500.0,Data Scientist
2,562599,75500.0,Data Scientist
3,116674,85000.0,Data Scientist
4,116674,85000.0,Data Scientist
...,...,...,...
653,98865,81000.0,Data Engineer
654,79066,98500.0,Data Engineer
655,309031,121000.0,Data Scientist
656,121536,130000.0,Data Scientist


In [129]:
df3 = pd.get_dummies(df3)
df3

,Population,Salary,Categorized Job Title_Data Analyst,Categorized Job Title_Data Engineer,Categorized Job Title_Data Scientist,Categorized Job Title_Other
0,562599,72000.0,0,0,1,0
1,562599,75500.0,0,0,1,0
2,562599,75500.0,0,0,1,0
3,116674,85000.0,0,0,1,0
4,116674,85000.0,0,0,1,0
...,...,...,...,...,...,...
653,98865,81000.0,0,1,0,0
654,79066,98500.0,0,1,0,0
655,309031,121000.0,0,0,1,0
656,121536,130000.0,0,0,1,0


In [130]:

from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
#model = RandomForestClassifier()
#model =LinearRegression()
model = RandomForestRegressor()

y=np.array(df3['Salary']).reshape(-1,1)
x=np.array(df3.drop(columns={"Salary"}))

xtrain, xtest, ytrain, ytest = train_test_split(x, y)
model.fit(xtrain, ytrain.ravel())
pred=model.predict(xtest)
print(mean_squared_error(pred, ytest, squared=False))



31731.852597531197


In [131]:
location_to_population_converter_list=[]

In [132]:
data= {
          'Population':[df2.loc[df2['Location']=='Boston']['Population'].item()]
          }
user_df = pd.DataFrame(data)


user_df['Data Analyst'] = 1
user_df['Data Engineer'] = 0
user_df['Data Scientist'] = 0
user_df['Other'] = 0

user_data = user_df['Population'].values.reshape(-1, 1)

variable= model.predict(user_df)

user_df['Salary Prediction']=variable
user_df

c:\Users\billg\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


,Population,Data Analyst,Data Engineer,Data Scientist,Other,Salary Prediction
0,654776,1,0,0,0,71007.145833
